In [38]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id='cid', client_secret='c_secret')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [39]:
playlist_link = "https://open.spotify.com/playlist/5JFforWrze2uuzHMYMygIN"
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_link)["items"]]

In [40]:
for track in sp.playlist_tracks(playlist_link)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]

# Extracting features from the tracks using track_uri obtained

In [41]:
sp.audio_features(track_uri)[0]

{'danceability': 0.633,
 'energy': 0.507,
 'key': 11,
 'loudness': -6.779,
 'mode': 0,
 'speechiness': 0.0456,
 'acousticness': 0.206,
 'instrumentalness': 2.97e-05,
 'liveness': 0.358,
 'valence': 0.367,
 'tempo': 75.04,
 'type': 'audio_features',
 'id': '2yk8XLalqQWjBd8GLBq7kS',
 'uri': 'spotify:track:2yk8XLalqQWjBd8GLBq7kS',
 'track_href': 'https://api.spotify.com/v1/tracks/2yk8XLalqQWjBd8GLBq7kS',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2yk8XLalqQWjBd8GLBq7kS',
 'duration_ms': 229983,
 'time_signature': 4}

# Features 